In [1]:
import pandas as pd
import numpy as np

# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [2]:
# carga datos diádicos
pbi = cargar_pbi()
md = cargar_matriz_distancias()
vecindad = cargar_vecindad()
dist_loc = pd.read_csv('tablas/df_distancias_localidades.csv', usecols=['cod','distancia_m'])

# carga capas de datos geográficos
deptos, deptos_sim, localidad, centro_pobl, capital = cargar_datos_geo()

# carga migrantes internos
p_migr = cargar_migrantes_internos()

# carga poblacion por depto
pobl = recuperar_poblacion_2011()

# menores de 25 años por depto
censo_edad = cargar_censo_vars(['DPTO', 'PERNA01'])
censo_edad = censo_edad.loc[censo_edad.PERNA01 < 25].reset_index(drop=True)
menores_25 = censo_edad.groupby('DPTO').size()
menores_25.name = 'menores_25'

# carga cantidad de empresas industriales por depto (OTU, OPP, 2013)
# Fuente: Núcleo Interdisciplinario de Estudios de Desarrollo Territorial (NIEDT)
# en base a INE (Directorio de Empresas y Establecimientos) y relevamiento propio
# Base: Total de empresas industriales de más de 5 empleados
empresas = pd.read_csv('tablas/empresas_por_depto.csv', sep=';', usecols=['DPTO', 'empresas'])

In [3]:
# genera código de díada
p_migr['cod'] = (p_migr['loc_origen'].astype(str) + p_migr['loc_destino'].astype(str)).astype(int)

# pega distancias entre localidades
p_migr = p_migr.merge(dist_loc, how='left', on='cod')
p_migr.head(2)

print(p_migr.shape)
print(p_migr.columns)
p_migr.head()

(148759, 53)
Index(['depto_origen', 'depto_destino', 'LOC', 'SECC', 'SEGM', 'VIVID',
       'TIPO_VIVIE', 'HOGCOD', 'HOGID', 'PERPH02', 'PERPA01', 'PERER02',
       'PERNA01', 'PERNA02', 'PERMI01', 'PERMI01_1', 'PERMI01_2', 'PERMI01_3',
       'PERMI01_4', 'PERMI02', 'PERMI02_1', 'PERMI05', 'PERMI05_1', 'PERMI06',
       'PERMI06_1', 'PERMI06_2', 'PERMI06_3', 'PERMI06_4', 'PERMI07',
       'PERMI07_1', 'PERMI07_2', 'PERMI07_3', 'PERMI07_4', 'PERED00',
       'PERED01', 'PERED02', 'PERED02_1', 'PERED02_2', 'PERED02_3',
       'PERED02_4', 'PERED03_R', 'PERED03_1', 'PERED03_2', 'PERED04_R',
       'PERED05_R', 'PERED06_R', 'CODIGO_CAR', 'PERED08', 'NIVELEDU_R',
       'loc_origen', 'loc_destino', 'cod', 'distancia_m'],
      dtype='object')


,depto_origen,depto_destino,LOC,SECC,SEGM,VIVID,TIPO_VIVIE,HOGCOD,HOGID,PERPH02,...,PERED04_R,PERED05_R,PERED06_R,CODIGO_CAR,PERED08,NIVELEDU_R,loc_origen,loc_destino,cod,distancia_m
0,3,1,20,1,1,5,1,102001001000501,1,1,...,2,3,4,55131,1,9,3732,1020,37321020,25034.0
1,17,1,20,1,1,30,1,102001001003001,1,2,...,0,2,4,55152,1,9,17220,1020,172201020,265706.0
2,18,1,20,1,1,36,1,102001001003601,1,1,...,2,2,0,33101,1,5,18220,1020,182201020,379469.0
3,2,1,20,1,1,46,1,102001001004601,1,1,...,2,1,0,33101,1,5,2220,1020,22201020,583715.0
4,12,1,20,1,1,93,1,102001001009301,1,2,...,2,1,0,22101,1,4,12320,1020,123201020,297600.0


In [4]:
p_migr.loc[(p_migr.depto_origen==1) & (p_migr.depto_destino==3), 'distancia_m'].max()

85762.0

In [5]:
p_migr.loc[(p_migr.depto_origen==3) & (p_migr.depto_destino==1), 'distancia_m'].max()

85762.0

In [6]:
# genera un sólo dataframe solo para deptos
flujos_deptos = p_migr[['depto_origen', 'depto_destino', 'PERNA01']].copy()

flujos_deptos = flujos_deptos[flujos_deptos.depto_origen != flujos_deptos.depto_destino]

flujos_deptos['personas_mig'] = 1
flujos_deptos['personas_mig_18_25'] = 0
flujos_deptos['personas_mig_26_35'] = 0

flujos_deptos.loc[flujos_deptos.PERNA01.between(18, 25), 'personas_mig_18_25'] = 1
flujos_deptos.loc[flujos_deptos.PERNA01.between(26, 35), 'personas_mig_26_35'] = 1

flujos_deptos.drop('PERNA01', axis=1, inplace=True)


# agrupa y cuenta
grupo = flujos_deptos.groupby(by=['depto_origen', 'depto_destino']).sum()

print('Cantidad de díadas ', grupo.shape[0])

grupo.head(3)

Cantidad de díadas  342


personas_mig  personas_mig_18_25  \
depto_origen depto_destino                                     
1            2                       914                 114   
             3                     33127                3757   
             4                      1387                 162   

                            personas_mig_26_35  
depto_origen depto_destino                      
1            2                             247  
             3                            7904  
             4                             401

In [7]:
data_webmap = grupo.reset_index().loc[:, ['depto_origen', 'depto_destino', 'personas_mig']]
data_webmap.to_csv('webmap_flows/flows.csv', index=False)

In [8]:
# flujos en movimientos mayores a 100km
flujos_deptos_100 = p_migr[['depto_origen', 'depto_destino', 'distancia_m']].copy()

flujos_deptos_100 = flujos_deptos_100[flujos_deptos_100.depto_origen != flujos_deptos_100.depto_destino]

flujos_deptos_100['personas_mig'] = 1

# agrupa y cuenta
grupo_100 = pd.DataFrame(flujos_deptos_100.loc[flujos_deptos_100.distancia_m > 100000].groupby(by=['depto_origen', 'depto_destino']).sum())

print('Cantidad de díadas ', grupo_100.shape[0])

grupo_100.head(3)

Cantidad de díadas  340


distancia_m  personas_mig
depto_origen depto_destino                           
1            2              524698229.0           891
             4              519691641.0          1349
             5              342413595.0          1991

In [9]:
# genera tabla pivot con los flujos de departamento a departamento
matrix = pd.pivot_table(flujos_deptos,
                        index ='depto_origen',
                        columns='depto_destino',
                        fill_value=0,
                        aggfunc=sum,
                        margins=True,
                        margins_name='Total')

In [10]:
# # guarda en csv
# matrix = pd.pivot_table(flujos_deptos,
#                         index ='depto_origen',
#                         columns='depto_destino',
#                         fill_value=0,
#                         aggfunc=sum,
#                         margins=False)

# matrix.to_csv('tablas/matriz_deptos.csv', header=True)

# # guarda en latex
# # lista de nombres deptos para generar tabla en latex
# nomdep = ['Mvdeo.', 'Artigas', 'Can.', 'C. Largo', 'Colonia', 'Durazno',
#           'Flores', 'Florida', 'Lavalleja','Maldonado', 'Paysandú', 'R. Negro', 'Rivera',
#           'Rocha', 'Salto', 'San José', 'Soriano', 'Tacuarembó', 'T. y Tres']

# coddep = np.arange(1, 20, 1).tolist()

# # guarda tabla en latex
# # cabecera = datos_dpto.sort_values('DPTO')['NOMBRE'].to_list()
# # cabecera.append('Total')

# matrix_tex = matrix
# matrix_tex.rename(index=dict(zip(coddep, nomdep)), inplace=True)

# # setea ancho de columnas
# ancho = 'p{0.7cm}'
# colformato='l' + ancho * 20

# matrix_tex.to_latex(buf= "tablas/matriz_orig_dest.tex", bold_rows=False,
#                     column_format = colformato,
#                     caption= 'Matriz de movimientos entre departamentos (Censo INE 2011).')

In [11]:
# unimos todo en un dataframe de díadas

# migrantes
df_agrupado = grupo.reset_index()

# agrega codigo único
df_agrupado.insert(0, 'cod', (df_agrupado['depto_origen'].astype(str)
                   + df_agrupado['depto_destino'].astype(str).str.zfill(2)).astype(int))

df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35
0,102,1,2,914,114,247
1,103,1,3,33127,3757,7904


In [12]:
df_agrupado_100k = grupo_100.reset_index()
# agrega codigo único
df_agrupado_100k.insert(0, 'cod', (df_agrupado_100k['depto_origen'].astype(str)
                   + df_agrupado_100k['depto_destino'].astype(str).str.zfill(2)).astype(int))

df_agrupado_100k.drop(['depto_origen', 'depto_destino', 'distancia_m'], axis=1, inplace=True)
df_agrupado_100k.rename({'personas_mig': 'personas_mig_100k'}, axis=1, inplace=True)
df_agrupado_100k.head(2)

,cod,personas_mig_100k
0,102,891
1,104,1349


In [13]:
# pega
df_agrupado = df_agrupado.merge(df_agrupado_100k, how='left', on='cod')

df_agrupado['personas_mig_100k'] = df_agrupado['personas_mig_100k'].fillna(0).astype(int)

df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k
0,102,1,2,914,114,247,891
1,103,1,3,33127,3757,7904,0


In [14]:
# pega masas (población!)
df_agrupado = df_agrupado.merge(pobl, how='left', left_on='depto_origen', right_on='DPTO')

df_agrupado = df_agrupado.merge(pobl, how='left', left_on='depto_destino', right_on='DPTO')

def drop_deptos(df):
    df = df.drop(['DPTO_x', 'DPTO_y'], axis=1, inplace=True)

drop_deptos(df_agrupado)

df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,poblacion_x,poblacion_y
0,102,1,2,914,114,247,891,1318755,73377
1,103,1,3,33127,3757,7904,0,1318755,520173


In [15]:
# pega matriz de distancias
df_agrupado = df_agrupado.merge(md, how='left', on='cod')
df_agrupado = df_agrupado.drop(['cod_ori', 'cod_des'], axis=1)

# convierte distancia a KM
df_agrupado['dist_km'] = round(df_agrupado.distancia / 1000).astype(int)

df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,poblacion_x,poblacion_y,distancia,dist_km
0,102,1,2,914,114,247,891,1318755,73377,610726,611
1,103,1,3,33127,3757,7904,0,1318755,520173,22544,23


In [16]:
# pega datos del PBI
df_agrupado = df_agrupado.merge(pbi, how='left', left_on='depto_origen',  right_on='DPTO')
df_agrupado = df_agrupado.merge(pbi, how='left', left_on='depto_destino', right_on='DPTO')

# convierte unidades
df_agrupado['pbi_origen_millardos']  = round(df_agrupado.miles_de_pesos_x / 1000000, 3)
df_agrupado['pbi_destino_millardos'] = round(df_agrupado.miles_de_pesos_y / 1000000, 3)

drop_deptos(df_agrupado)

df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,poblacion_x,poblacion_y,distancia,dist_km,miles_de_pesos_x,porcentaje_pbi_x,miles_de_pesos_y,porcentaje_pbi_y,pbi_origen_millardos,pbi_destino_millardos
0,102,1,2,914,114,247,891,1318755,73377,610726,611,465848031,50.3,14214980,1.5,465.848,14.215
1,103,1,3,33127,3757,7904,0,1318755,520173,22544,23,465848031,50.3,86306492,9.3,465.848,86.306


In [17]:
# pega vecindad
df_agrupado = df_agrupado.merge(vecindad, how='left', on='cod').drop(['dep1', 'dep2'], axis=1)
df_agrupado['dummy_limit'] = True
df_agrupado.loc[df_agrupado.largo_limite.isnull(), 'dummy_limit'] = False

df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,poblacion_x,poblacion_y,distancia,dist_km,miles_de_pesos_x,porcentaje_pbi_x,miles_de_pesos_y,porcentaje_pbi_y,pbi_origen_millardos,pbi_destino_millardos,largo_limite,dummy_limit
0,102,1,2,914,114,247,891,1318755,73377,610726,611,465848031,50.3,14214980,1.5,465.848,14.215,NaN,False
1,103,1,3,33127,3757,7904,0,1318755,520173,22544,23,465848031,50.3,86306492,9.3,465.848,86.306,60107.0,True


In [18]:
# pega empresas
df_agrupado = df_agrupado.merge(empresas, how='left', left_on='depto_origen', right_on='DPTO')
df_agrupado = df_agrupado.merge(empresas, how='left', left_on='depto_destino', right_on='DPTO')

drop_deptos(df_agrupado)

df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,poblacion_x,poblacion_y,distancia,...,miles_de_pesos_x,porcentaje_pbi_x,miles_de_pesos_y,porcentaje_pbi_y,pbi_origen_millardos,pbi_destino_millardos,largo_limite,dummy_limit,empresas_x,empresas_y
0,102,1,2,914,114,247,891,1318755,73377,610726,...,465848031,50.3,14214980,1.5,465.848,14.215,NaN,False,3124,55
1,103,1,3,33127,3757,7904,0,1318755,520173,22544,...,465848031,50.3,86306492,9.3,465.848,86.306,60107.0,True,3124,563


In [19]:
# pega menores de 25 años
df_agrupado = df_agrupado.merge(menores_25, how='left', left_on='depto_origen', right_on='DPTO')
df_agrupado = df_agrupado.merge(menores_25, how='left', left_on='depto_destino', right_on='DPTO')
df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,poblacion_x,poblacion_y,distancia,...,miles_de_pesos_y,porcentaje_pbi_y,pbi_origen_millardos,pbi_destino_millardos,largo_limite,dummy_limit,empresas_x,empresas_y,menores_25_x,menores_25_y
0,102,1,2,914,114,247,891,1318755,73377,610726,...,14214980,1.5,465.848,14.215,NaN,False,3124,55,456072,30880
1,103,1,3,33127,3757,7904,0,1318755,520173,22544,...,86306492,9.3,465.848,86.306,60107.0,True,3124,563,456072,197993


In [20]:
dict_rename = {
    'miles_de_pesos_x':'pbi_origen',
    'miles_de_pesos_y':'pbi_destino',
    'porcentaje_pbi_x': 'pbi_porcen_ori',
    'porcentaje_pbi_y': 'pbi_porcen_des',
    'empresas_x':'empresas_origen',
    'empresas_y': 'empresas_destino',
    'miles_de_pesos':'pbi_destino',
    'distancia': 'dist',
    'poblacion_x': 'pob_origen',
    'poblacion_y': 'pob_destino',
    'menores_25_x': 'menores_25_origen',
    'menores_25_y': 'menores_25_destino'
}

df_agrupado.rename(dict_rename, inplace=True, axis=1)

df_agrupado.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,pob_origen,pob_destino,dist,...,pbi_destino,pbi_porcen_des,pbi_origen_millardos,pbi_destino_millardos,largo_limite,dummy_limit,empresas_origen,empresas_destino,menores_25_origen,menores_25_destino
0,102,1,2,914,114,247,891,1318755,73377,610726,...,14214980,1.5,465.848,14.215,NaN,False,3124,55,456072,30880
1,103,1,3,33127,3757,7904,0,1318755,520173,22544,...,86306492,9.3,465.848,86.306,60107.0,True,3124,563,456072,197993


In [21]:
# genera otro DF igual a df_agrupado, pero llamado dd_deptos
dd_deptos = df_agrupado

print(dd_deptos.shape)

(342, 23)


In [22]:
## pob destino y origen a miles
dd_deptos['pob_destino_k'] = dd_deptos['pob_destino']/1000
dd_deptos['pob_origen_k'] = dd_deptos['pob_origen']/1000

In [23]:
# decodifica el departamento (códigos INE)
dd_deptos['nom_depto_orig'] = decode_depto(dd_deptos, 'depto_origen')
dd_deptos['nom_depto_des'] = decode_depto(dd_deptos, 'depto_destino')

dd_deptos.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,pob_origen,pob_destino,dist,...,largo_limite,dummy_limit,empresas_origen,empresas_destino,menores_25_origen,menores_25_destino,pob_destino_k,pob_origen_k,nom_depto_orig,nom_depto_des
0,102,1,2,914,114,247,891,1318755,73377,610726,...,NaN,False,3124,55,456072,30880,73.377,1318.755,MONTEVIDEO,ARTIGAS
1,103,1,3,33127,3757,7904,0,1318755,520173,22544,...,60107.0,True,3124,563,456072,197993,520.173,1318.755,MONTEVIDEO,CANELONES


In [24]:
censo = cargar_censo_nrows(2)
censo.columns

Index(['DPTO', 'LOC', 'SECC', 'SEGM', 'VIVID', 'TIPO_VIVIE', 'HOGID',
       'PERPH02', 'PERPA01', 'PERER02', 'PERNA01', 'PERNA02', 'PERMI01',
       'PERMI01_1', 'PERMI01_2', 'PERMI01_3', 'PERMI01_4', 'PERMI02',
       'PERMI02_1', 'PERMI05', 'PERMI05_1', 'PERMI06', 'PERMI06_1',
       'PERMI06_2', 'PERMI06_3', 'PERMI06_4', 'PERMI07', 'PERMI07_1',
       'PERMI07_2', 'PERMI07_3', 'PERMI07_4', 'PERED00', 'PERED01', 'PERED02',
       'PERED02_1', 'PERED02_2', 'PERED02_3', 'PERED02_4', 'PERED03_R',
       'PERED03_1', 'PERED03_2', 'PERED04_R', 'PERED05_R', 'PERED06_R',
       'CODIGO_CAR', 'PERED08', 'NIVELEDU_R', 'POBPCOAC'],
      dtype='object')

In [25]:
# agrega edad promedio por departamento
cols = ['DPTO', 'PERNA01', 'POBPCOAC']
censo = cargar_censo_vars(cols)
censo.head()

,DPTO,PERNA01,POBPCOAC
0,1,22,6
1,1,84,5
2,1,21,4
3,1,0,1
4,1,31,2


In [26]:
emedia = censo.groupby('DPTO').mean()['PERNA01'].to_frame()
emedia.head(2)

,PERNA01
DPTO,
1,37.481048
2,33.966911


In [27]:
emedia = censo.groupby('DPTO').mean()['PERNA01'].to_frame()
emedia.head(2)

,PERNA01
DPTO,
1,37.481048
2,33.966911


In [28]:
ocupados = censo.loc[censo.POBPCOAC==2, 'DPTO'].to_frame()
ocup_gr = ocupados.groupby('DPTO').size()

pet = censo.loc[(censo.PERNA01 >= 15) & (censo.PERNA01 < 65), 'DPTO'].to_frame()
pet_gr = pet.groupby('DPTO').size().values

porc_ocupados = (ocup_gr / pet_gr)*100

In [29]:
emedia['porc_ocupados'] = porc_ocupados

In [30]:
dd_deptos_new = dd_deptos.merge(emedia, left_on='depto_origen', right_on='DPTO')
dd_deptos_new = dd_deptos_new.merge(emedia, left_on='depto_destino', right_on='DPTO')

dict_rename = {'PERNA01_x': 'edad_prom_ori',
               'porc_ocupados_x': 'porc_ocupados_ori',
               'PERNA01_y': 'edad_prom_des',
               'porc_ocupados_y': 'porc_ocupados_des'}

dd_deptos_new.rename(dict_rename, inplace=True, axis=1)

dd_deptos_new.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,pob_origen,pob_destino,dist,...,menores_25_origen,menores_25_destino,pob_destino_k,pob_origen_k,nom_depto_orig,nom_depto_des,edad_prom_ori,porc_ocupados_ori,edad_prom_des,porc_ocupados_des
0,102,1,2,914,114,247,891,1318755,73377,610726,...,456072,30880,73.377,1318.755,MONTEVIDEO,ARTIGAS,37.481048,70.395702,33.966911,63.540489
1,302,3,2,74,9,22,70,520173,73377,601915,...,197993,30880,73.377,520.173,CANELONES,ARTIGAS,35.406640,71.045180,33.966911,63.540489


In [31]:
#### Tasa de desempleo 2011
empleo = pd.read_csv('tablas/tasa-desempleo.csv', sep=';', decimal=',')

emp_m = empleo.loc[(empleo.Sexo=='Mujeres') & (empleo.Departamento!='Total'), '2011']
emp_v = empleo.loc[(empleo.Sexo=='Varones') & (empleo.Departamento!='Total'), '2011']
emp_mv = empleo.loc[(empleo.Sexo=='Total') & (empleo.Departamento!='Total'), '2011']
dep = empleo.loc[(empleo.Sexo=='Total') & (empleo.Departamento!='Total'), 'Departamento']
emp = [encode_depto_pretty(dep), emp_m, emp_v, emp_mv]
[i.reset_index(inplace=True, drop=True) for i in emp]

emp_concat = pd.concat(emp, axis=1)

emp_concat.head()


,Departamento,2011,2011,2011
0,1,7.4,5.1,6.2
1,2,11.5,5.4,8.1
2,3,8.1,4.3,6.0
3,4,3.8,1.7,2.6
4,5,4.1,2.5,3.2


In [32]:
emp_concat.columns = [
    'depto',
    'tasa_desemp_mujeres_2011_ori',
    'tasa_desemp_varones_2011_ori',
    'tasa_desemp_total_2011_ori'
]


In [33]:
dd_deptos_new = dd_deptos_new.merge(
    emp_concat, left_on='depto_origen', right_on='depto')

emp_concat.columns = ['depto',
                      'tasa_desemp_mujeres_2011_des',
                      'tasa_desemp_varones_2011_des',
                      'tasa_desemp_total_2011_des'
                      ]

dd_deptos_new = dd_deptos_new.merge(
    emp_concat, left_on='depto_destino', right_on='depto')
cols = ['depto_x', 'depto_y']
dd_deptos_new.drop(cols, axis=1, inplace=True)

dd_deptos_new.head()


,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,pob_origen,pob_destino,dist,...,edad_prom_ori,porc_ocupados_ori,edad_prom_des,porc_ocupados_des,tasa_desemp_mujeres_2011_ori,tasa_desemp_varones_2011_ori,tasa_desemp_total_2011_ori,tasa_desemp_mujeres_2011_des,tasa_desemp_varones_2011_des,tasa_desemp_total_2011_des
0,102,1,2,914,114,247,891,1318755,73377,610726,...,37.481048,70.395702,33.966911,63.540489,7.4,5.1,6.2,11.5,5.4,8.1
1,302,3,2,74,9,22,70,520173,73377,601915,...,35.406640,71.045180,33.966911,63.540489,8.1,4.3,6.0,11.5,5.4,8.1
2,402,4,2,15,0,6,14,84698,73377,406694,...,35.614749,65.582039,33.966911,63.540489,3.8,1.7,2.6,11.5,5.4,8.1
3,502,5,2,28,7,8,25,123203,73377,556292,...,37.632420,76.058415,33.966911,63.540489,4.1,2.5,3.2,11.5,5.4,8.1
4,602,6,2,32,0,10,29,57084,73377,446123,...,35.093021,67.284108,33.966911,63.540489,13.1,5.8,8.9,11.5,5.4,8.1


In [34]:
# calcula logaritmos
def logaritmos(df, variables):
    "Calcula logaritmos para N variables"
    new_vars = ['log_' + i for i in variables]
    n = len(variables)
    for i in range(n):
        df[new_vars[i]] = np.log(df[variables[i]])

    return df

# definir lista de variables sobre las cuales se quiere el logaritmo
variables = [
    'pbi_destino',
    'dist_km',
    'pbi_origen_millardos',
    'pbi_destino_millardos',
    'pob_destino_k',
    'pob_origen_k',
    'edad_prom_des',
    'porc_ocupados_des'
]

dd_deptos_new = logaritmos(dd_deptos_new, variables)

dd_deptos_new.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,pob_origen,pob_destino,dist,...,tasa_desemp_varones_2011_des,tasa_desemp_total_2011_des,log_pbi_destino,log_dist_km,log_pbi_origen_millardos,log_pbi_destino_millardos,log_pob_destino_k,log_pob_origen_k,log_edad_prom_des,log_porc_ocupados_des
0,102,1,2,914,114,247,891,1318755,73377,610726,...,5.4,8.1,16.469807,6.415097,6.143859,2.654298,4.295611,7.184443,3.525387,4.151677
1,302,3,2,74,9,22,70,520173,73377,601915,...,5.4,8.1,16.469807,6.400257,4.457899,2.654298,4.295611,6.254161,3.525387,4.151677


In [41]:
# carga empresas
pymes = pd.read_csv('tablas/mypymes.csv')
pymes.head()

,NOMBRE_DEPARTAMENTO,A,B,C,D,E,F,G,H,I,J,K,L,M,N,P,Q,R,S,total
0,ARTIGAS,36,65,279,0,8,97,1175,291,98,45,16,14,100,93,55,54,38,254,2718
1,CANELONES,225,45,2570,2,69,1360,8557,2189,1014,586,230,393,1211,1332,726,735,419,2096,23759
2,CERRO LARGO,53,3,275,1,5,109,1415,291,115,38,16,20,157,132,45,97,49,345,3166
3,COLONIA,121,20,865,3,24,332,3108,835,523,126,54,143,422,455,202,208,149,711,8301
4,DURAZNO,59,7,204,1,5,101,1135,206,119,45,17,26,95,96,40,90,36,229,2511


In [35]:
# guarda
dd_deptos_new.to_csv('tablas/dd_deptos.csv', index=False, sep=';', decimal=',')